# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [48]:
# Instalar o selenium
# Baixar o webdriver e colocar ele na mesma pasta do seu código
# Chrome --> chromedriver
# Firefox --> geckdriver

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

navegador = webdriver.Chrome('chromedriver.exe')


# Passo 1: Pegar a Cotação do Dólar
# Entrar no Site do Google
navegador.get('https://www.google.com')

# Pesquisar a Cotação do Dólar
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação Dólar')
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# Pegar a Cotação da Página do Google
cotacao_dolar = navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(' A Cotação do Dólar é de: R$ {}'.format(cotacao_dolar))

# Passo 2: Pegar a Cotação do Euro
navegador.get('https://www.google.com')
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação Euro')
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(' A Cotação do Euro é de: R$ {}'.format(cotacao_euro))
# Passo 3: Pegar a Cotação do Ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element(By.XPATH,'//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',','.')
print(' A Cotação do Ouro é de: R$ {}'.format(cotacao_ouro))


# pyautogui --> processos no seu computador (que não são no navegador)
# selenium --> processos no navegador

C:\Users\PC\AppData\Local\Temp\ipykernel_11676\2267382242.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  navegador = webdriver.Chrome('chromedriver.exe')


 A Cotação do Dólar é de: R$ 5.2656
 A Cotação do Euro é de: R$ 5.10341952
 A Cotação do Ouro é de: R$ 279.56


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [49]:
# Passo 4: Importar a Base de Dados 
import pandas as pd
tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [50]:
# Passo 5: Atualizar a Cotação, o Preço de Compra e o Preço de Venda
# Atualizar a Cotação
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

# Atualizar o Preço de Compra = Preço Original * Cotação
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']
# Atualizar o Preço de Venda = Preço de Compra * Margem
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']
#tabela['Preço de Venda'] = tabela['Preço de Venda'].map('{:.2f}'.format)
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.26560,5265.547344,1.40,7371.766282
1,Carro Renault,4500.00,Euro,5.10342,22965.387840,2.00,45930.775680
2,Notebook Dell,899.99,Dólar,5.26560,4738.987344,1.70,8056.278485
3,IPhone,799.00,Dólar,5.26560,4207.214400,1.70,7152.264480
4,Carro Fiat,3000.00,Euro,5.10342,15310.258560,1.90,29089.491264
5,Celular Xiaomi,480.48,Dólar,5.26560,2530.015488,2.00,5060.030976
6,Joia 20g,20.00,Ouro,279.56000,5591.200000,1.15,6429.880000


### Agora vamos exportar a nova base de preços atualizada

In [51]:
# Passo 6: Exportar o Relatório Atualizado
tabela.to_excel('Produtos Novo.xlsx', index=False)
navegador.quit()